In [17]:
import json 
import pandas as pd 

# Test model results 

In [18]:
test_model_results = []

with open("test_responses.jsonl",'r') as f : 
    for line in f: 
        if line.strip():
            data = json.loads(line)
            test_model_results.append(data)


In [19]:
df_test_model = pd.DataFrame(test_model_results)
df_test_model

,sample_id,test_model,Encountered_Problems,test_model_response
0,0,openai:gpt-3.5-turbo,False,Based on the information provided in the conte...
1,0,ollama:llama3.2,True,[Errno 61] Connection refused
2,1,openai:gpt-3.5-turbo,False,1. Wüsthof - 14 degrees\n2. MAC - 15 degrees\n...
3,1,ollama:llama3.2,True,[Errno 61] Connection refused


In [12]:
print(df_test_model['test_model_response'].iloc[0])

Based on the information provided in the context document, your risk factors for dementia include high blood pressure, diabetes, obesity, lack of physical activity, poor diet, high alcohol consumption, low cognitive engagement, depression, traumatic brain injury, hearing loss, and social isolation.

Cognitive engagement is important in reducing the risk of dementia because it is thought to support the development of a "cognitive reserve." This means that actively using your brain throughout your life may provide protection against brain cell damage caused by dementia. Therefore, engaging in activities that challenge and stimulate your brain can help reduce the risk of developing dementia.


In [13]:
print(df_test_model['test_model_response'].iloc[1])

[Errno 61] Connection refused


# Judges

In [20]:
judges_result = []

with open("judge_responses.jsonl",'r') as f : 
    for line in f:
        if line.strip():
            content = json.loads(line)
            judges_result.append(content)

In [21]:
df_judges = pd.DataFrame(judges_result)
df_judges

,judged_model,sample_id,judge_model,Encountered_Problems,judge_model_response,verdict
0,openai:gpt-3.5-turbo,0,anthropic:claude-3-7-sonnet-20250219,False,Sentence 1: Based on the information provided ...,Inaccurate
1,openai:gpt-3.5-turbo,0,ollama:llama3.2,True,[Errno 61] Connection refused,None
2,ollama:llama3.2,0,anthropic:claude-3-7-sonnet-20250219,Not applicable,Not applicable,Not applicable
3,ollama:llama3.2,0,ollama:llama3.2,Not applicable,Not applicable,Not applicable
4,openai:gpt-3.5-turbo,1,anthropic:claude-3-7-sonnet-20250219,False,Sentence 1: Wüsthof - 14 degrees\nSentence 1 l...,Inaccurate
5,openai:gpt-3.5-turbo,1,ollama:llama3.2,True,[Errno 61] Connection refused,None
6,ollama:llama3.2,1,anthropic:claude-3-7-sonnet-20250219,Not applicable,Not applicable,Not applicable
7,ollama:llama3.2,1,ollama:llama3.2,Not applicable,Not applicable,Not applicable


In [7]:
print(len(df_test_model))
print(len(df_judges))

4
8


# Explore some samples 

In [8]:
print(df_judges['judge model response'].iloc[0])

Sentence 1: Based on the information provided in the context document, your risk factors for dementia include high blood pressure, diabetes, obesity, lack of physical activity, poor diet, high alcohol consumption, low cognitive engagement, depression, traumatic brain injury, hearing loss, and social isolation.
Sentence 1 label: Inaccurate

Sentence 2: Cognitive engagement is important because it is thought to support the development of a "cognitive reserve," which may protect against brain cell damage caused by dementia.
Sentence 2 label: Accurate

Sentence 3: Engaging in activities that challenge and stimulate the brain throughout life may help reduce the risk of developing dementia.
Sentence 3 label: Accurate

Final Answer: Inaccurate

The first sentence is inaccurate because it lists all possible risk factors from the evidence as applying to the person in the query, when in fact the query only mentions some of these factors. The person states they "never smoked" (so smoking is not a

In [9]:
print(df_judges['judge model response'].iloc[1])

{"sentence": "I'm middle-aged, never smoked, had my ears blown out in the war, get a case of the sads pretty regular, and eat mostly garbage.", "label": "no_rad", "rationale": "This is a personal statement and does not require factual attribution.", "excerpt": null}
{"sentence": "What are my risk factors for dementia?", "label": "unsupported", "rationale": "The question is asking for specific information, but the context only provides general information about risk factors for dementia.", "excerpt": null}
{"sentence": "What does cognitive engagement have to do with it?", "label": "no_rad", "rationale": "This is a follow-up question and does not require factual attribution.", "excerpt": null}
{"sentence": "Based on the information provided in the context document, your risk factors for dementia include high blood pressure, diabetes, obesity, lack of physical activity, poor diet, high alcohol consumption, low cognitive engagement, depression, traumatic brain injury, hearing loss, and soc

# Overall Accuracy 

In [10]:
# TEST_MODEL = 'openai:gpt-3.5-turbo' # feel free to checkout accuracy for other models 
# TEST_MODEL = 'openai:gpt-4o' # feel free to checkout accuracy for other models 
TEST_MODEL = 'ollama:llama3.2'

In [11]:
df_judges[df_judges['judged_model']==TEST_MODEL].groupby('judge model')['verdict'].value_counts().unstack(fill_value=0)

verdict,Inaccurate
judge model,
anthropic:claude-3-7-sonnet-20250219,2
ollama:llama3.2,2


In [12]:
df_filtered = df_judges[df_judges['judged_model'] == TEST_MODEL]
counts = df_filtered.groupby('judge model')['verdict'].value_counts()
percentages = counts / counts.groupby(level=0).sum() * 100
result = percentages.unstack(fill_value=0)[["Accurate"]]

result

KeyError: "None of [Index(['Accurate'], dtype='object', name='verdict')] are in the [columns]"